In [ ]:
from policyengine_us import Microsimulation
from policyengine_core.model_api import *

cgt_reform = Reform.from_dict({
    "gov.simulation.capital_gains_responses.elasticity": -0.7,
    "gov.irs.capital_gains.brackets.rates.3": 0.39,
})

sim = Microsimulation(reform=cgt_reform)

In [ ]:
sim